In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import weather_api_key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/city_weather_data.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.6001,-73.9662,0.23,74,46,0.89,US,Wed Feb 21 22:10:59 2024
1,bilibino,68.0546,166.4372,-29.07,91,97,3.39,RU,Wed Feb 21 22:12:32 2024
2,edinburgh of the seven seas,-37.0676,-12.3116,18.65,92,100,12.36,SH,Wed Feb 21 22:12:33 2024
3,vila velha,-20.3297,-40.2925,24.97,94,100,5.81,BR,Wed Feb 21 22:12:33 2024
4,troy,42.6056,-83.1499,11.80,62,40,2.06,US,Wed Feb 21 22:12:33 2024


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
import pandas as pd
import hvplot.pandas  # Make sure hvplot is installed

# Assuming city_data_df is your DataFrame and it has columns 'City', 'Lat', 'Lng', 'Humidity'
# Make sure your DataFrame is loaded correctly
city_data_df = pd.read_csv('output_data/city_weather_data.csv')  # Uncomment and modify if needed

# Create a map using hvplot with points sized by humidity
# Adjust the size factor in the `size` argument as needed to scale the point sizes appropriately
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size=city_data_df['Humidity']*0.1,  # Scale size by humidity
                                      hover_cols=['City'], color='Humidity', cmap='blues', alpha=0.6,
                                      tiles='OSM', frame_width=900, frame_height=600)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,_size,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0) ]
    

filtered_cities_df = filtered_cities_df.dropna()
filtered_cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,san pedro pochutla,15.7333,-96.4667,22.65,71,0,1.53,MX,Wed Feb 21 22:12:40 2024
23,puerto madero,14.7208,-92.4215,23.11,88,0,0.00,MX,Wed Feb 21 22:12:41 2024
41,mirpur sakro,24.5474,67.6277,22.48,16,0,4.30,PK,Wed Feb 21 22:12:49 2024
160,san luis de la loma,17.2706,-100.8939,24.47,66,0,2.26,MX,Wed Feb 21 22:13:26 2024
167,hawaiian paradise park,19.5933,-154.9731,22.73,77,0,2.06,US,Wed Feb 21 22:13:28 2024
196,brisas de zicatela,15.8369,-97.0419,24.49,68,0,1.72,MX,Wed Feb 21 22:13:34 2024
243,punta del este,-34.9667,-54.9500,21.04,88,0,2.68,UY,Wed Feb 21 22:13:47 2024
245,tolanaro,-25.0319,46.9987,24.04,85,0,3.71,MG,Wed Feb 21 22:13:48 2024
282,cabo san lucas,22.8909,-109.9124,21.53,70,0,4.24,MX,Wed Feb 21 22:05:44 2024
306,hilo,19.7297,-155.0900,22.85,77,0,2.06,US,Wed Feb 21 22:14:01 2024


Create a new DataFrame called hotel_df to store the city, country, coordinates, and humidity.

### Step 3: Create a new DataFrame called `hotel_df`.

In [18]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
# Selecting specific columns to copy into the new DataFrame
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Optionally, you can add an empty column for hotel information if you plan to fill it later
hotel_df['Hotel Name'] = ''
# Display the first few rows of the new DataFrame to confirm its structure
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,san pedro pochutla,MX,15.7333,-96.4667,71,
23,puerto madero,MX,14.7208,-92.4215,88,
41,mirpur sakro,PK,24.5474,67.6277,16,
160,san luis de la loma,MX,17.2706,-100.8939,66,
167,hawaiian paradise park,US,19.5933,-154.9731,77,


For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates.

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
import requests
import pandas as pd

# Assuming hotel_df is defined with columns ['City', 'Country', 'Lat', 'Lng']

# Set the search radius, in meters
# Set base URL for Geoapify Places API
base_url = "https://api.geoapify.com/v2/places"

print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Define API parameters for each city
    params = {
        "filter":f'circle:{row.Lng},{row.Lat},30000',
        "categories": "accommodation.hotel",
        "bias": f"proximity:{row.Lng},{row.Lat}",
        "limit": 5,  # Only fetch the first result
        "apiKey": geoapify_key  # Replace with your actual Geoapify API key
    }

    # Make the API request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract the hotel name from the first result, if available
        if data['features']:
            hotel_name = data['features'][0]['properties']['name']
            hotel_df.loc[index, "Hotel Name"] = hotel_name
            print(f"{row['City']} - nearest hotel: {hotel_name}")
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
            print(f"{row['City']} - No hotel found")
    else:
        print(f"Failed to fetch data for {row['City']}, status code: {response.status_code}")
        hotel_df.loc[index, "Hotel Name"] = "API request failed"

# Display a sample of the DataFrame to verify the results
hotel_df.dropna(inplace=True)
hotel_df.head()



Starting hotel search
san pedro pochutla - nearest hotel: Olimpia
puerto madero - nearest hotel: Hospedaje Villasana
mirpur sakro - No hotel found
san luis de la loma - No hotel found
hawaiian paradise park - nearest hotel: Lava Tree Tropic Inn
brisas de zicatela - nearest hotel: Casa de Olas
punta del este - nearest hotel: Hotel Aquarium
tolanaro - nearest hotel: Hôtel Mahavokey
cabo san lucas - nearest hotel: Comfort Rooms
hilo - nearest hotel: Dolphin Bay Hotel
la trinite - nearest hotel: Hôtel Le Manguier
toliara - nearest hotel: Ambary
ciudad melchor muzquiz - nearest hotel: Centro Express
francistown - nearest hotel: City Inn
coxen hole - nearest hotel: Elizabeth Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
20,san pedro pochutla,MX,15.7333,-96.4667,71,Olimpia
23,puerto madero,MX,14.7208,-92.4215,88,Hospedaje Villasana
41,mirpur sakro,PK,24.5474,67.6277,16,No hotel found
160,san luis de la loma,MX,17.2706,-100.8939,66,No hotel found
167,hawaiian paradise park,US,19.5933,-154.9731,77,Lava Tree Tropic Inn


In [29]:

hotel_df = hotel_df[hotel_df['Hotel Name']!='No hotel found']

In [30]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,san pedro pochutla,MX,15.7333,-96.4667,71,Olimpia
23,puerto madero,MX,14.7208,-92.4215,88,Hospedaje Villasana
167,hawaiian paradise park,US,19.5933,-154.9731,77,Lava Tree Tropic Inn
196,brisas de zicatela,MX,15.8369,-97.0419,68,Casa de Olas
243,punta del este,UY,-34.9667,-54.9500,88,Hotel Aquarium
245,tolanaro,MG,-25.0319,46.9987,85,Hôtel Mahavokey
282,cabo san lucas,MX,22.8909,-109.9124,70,Comfort Rooms
306,hilo,US,19.7297,-155.0900,77,Dolphin Bay Hotel
328,la trinite,MQ,14.7376,-60.9629,94,Hôtel Le Manguier
329,toliara,MG,-23.3500,43.6667,75,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

import hvplot.pandas  # for plotting with hvplot
import geopandas as gpd  # for geographical operations

# Assuming hotel_df is your DataFrame and it already contains 'Lat', 'Lng', 'City', 'Country', and 'Hotel Name' columns

# Convert your DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))

# Define columns to display on hover
hover_cols = ['City', 'Country', 'Hotel Name']

# Plot using hvplot with hover information
map_plot = gdf.hvplot(
    geo=True, 
    hover_cols=hover_cols, 
    tiles='OSM', 
    color='Hotel Name',  # This will use different colors for different hotels
    cmap='Category20',  # Color map for distinguishing hotels
    alpha=0.5,  # Transparency of the points
    size=200,  # Size of the points
    frame_width=700, 
    frame_height=500,
    title='Hotels Map'
)

# Enable hover tool
map_plot.opts(tools=['hover'])

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Hotel Name,City,Country)